The purpose of this notebook is to create and save a UMAP plot which shows the batch effect at the dataset level from a QClus filtered adata object.

In [1]:
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import warnings
#import harmonypy as hm

warnings.filterwarnings('ignore')

sc.set_figure_params(figsize=(8, 8), dpi=80, dpi_save=600)

In [2]:
datasets = ['Lit', 'Tu', 'Chaff', 'Koenig', 'kuopio', 'Hill']
method = "qclus"
all_adatas = []

for dataset in datasets:
    # Load the AnnData object
    adata = sc.read_h5ad(f'../scCAD/jupyter/qclus/benchmarking/qclus_results/dataset_adatas/{dataset}_dataset.h5ad')
    adata = adata.raw.to_adata()
    
    adata.obs['dataset'] = dataset
    
    # Load the processed_obs for qclus method
    processed_obs = pd.read_csv(f'../scCAD/jupyter/qclus/benchmarking/qclus_results/processing_results/{dataset}_dataset/comp1/{method}_processed_obs.csv', index_col=0)
    processed_obs.index = processed_obs.index.map(lambda x: '_'.join(x.split('_')[2:]))

    # Filter the adata object to include only the cells in processed_obs
    adata_filtered = adata[adata.obs.index.isin(processed_obs.index)]
    
    # Add the filtered adata to the list
    all_adatas.append(adata_filtered)#

# Concatenate all filtered adatas
adata = sc.concat(all_adatas, join='outer')

# Now global_adata is your concatenated AnnData object
adata

AnnData object with n_obs × n_vars = 1925265 × 36601
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'pct_counts_MT', 'leiden_dataset_before_harmony', 'leiden_dataset_after_harmony', 'leiden_dataset_3', 'score_MT', 'total_counts_MT_nucl', 'pct_counts_MT_nucl', 'score_MT_nucl', 'total_counts_CM_cyto', 'pct_counts_CM_cyto', 'score_CM_cyto', 'total_counts_CM_nucl', 'pct_counts_CM_nucl', 'score_CM_nucl', 'total_counts_VEC', 'pct_counts_VEC', 'score_VEC', 'total_counts_PER', 'pct_counts_PER', 'score_PER', 'total_counts_SMC', 'pct_counts_SMC', 'score_SMC', 'total_counts_AD', 'pct_counts_AD', 'score_AD', 'total_counts_SC', 'pct_counts_SC', 'score_SC', 'total_counts_N', 'pct_counts_N', 'score_N', 'total_counts_EEC', 'pct_counts_EEC', 'score_EEC', 'total_counts_FB', 'pct_counts_FB', 'score_FB', '

In [3]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_MT'], n_jobs=48)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)

KeyboardInterrupt: 

In [ ]:
datasets = ['Lit', 'Tu', 'Chaff', 'Koenig', 'kuopio', 'Hill']
dataset_names_corrected = ['Litviňuková et al.', 'CAREBANK', 'Chaffin et al.', 'Koenig et al.', 'PERIHEART', 'Hill et al.']

# Create a mapping dictionary
mapping_dict = dict(zip(datasets, dataset_names_corrected))

# Replace values in adata.obs['dataset']
adata.obs['dataset'] = adata.obs['dataset'].map(mapping_dict)

In [ ]:
sc.pl.umap(adata, color=['dataset', 'sample_id', 'cell_type'], save='_dataset_before_integration.png')